**⭐ 1. What This Pattern Solves**

Consolidates overlapping or adjacent intervals in datasets (time ranges, log windows, booking slots).

Common in ETL, analytics, and event pipelines where ranges must not overlap.

Useful for optimizing storage, computing coverage, or summarizing continuous periods.

**⭐ 2. SQL Equivalent**

In [0]:
%sql
WITH ordered AS (
    SELECT start, end
    FROM intervals
    ORDER BY start
)
SELECT MIN(start) AS merged_start,
       MAX(end) AS merged_end
FROM ordered
GROUP BY grp
-- 'grp' can be created using a window function to detect overlapping chains

**⭐ 3. Core Idea**

Sort intervals by start time, then merge overlapping ranges in one pass.

Keeps a rolling "current interval" to compare and extend.

**⭐ 4. Template Code (MEMORIZE THIS)**

In [0]:
def merge_intervals(intervals):
    if not intervals:
        return []
    intervals.sort(key=lambda x: x[0])
    merged = [intervals[0]]
    for start, end in intervals[1:]:
        last_end = merged[-1][1]
        if start <= last_end:  # Overlap
            merged[-1][1] = max(last_end, end)
        else:
            merged.append([start, end])
    return merged

**⭐ 5. Detailed Example**

In [0]:
#Input:

intervals = [[1,3], [2,6], [8,10], [9,12]]

merged = merge_intervals(intervals)
print(merged)


#Output:

[[1,6], [8,12]]

Explanation:

[1,3] overlaps with [2,6] → merge [1,6]

[8,10] overlaps with [9,12] → merge [8,12]

**⭐ 6. Mini Practice Problems**

Merge [[5,7],[1,3],[2,4],[6,8]].

Merge [[1,2],[3,4],[5,6]].

Merge [[1,10],[2,3],[4,5],[6,7]].

**⭐ 7. Full Data Engineering Scenario**

Problem Statement:

Logs contain session start and end times. Some sessions overlap. You need consolidated session periods per user to compute total active time.

Expected Output:

List of merged session intervals per user.

In [0]:
from collections import defaultdict

user_sessions = defaultdict(list)
for user, start, end in logs:
    user_sessions[user].append([start, end])

merged_sessions = {}
for user, intervals in user_sessions.items():
    merged_sessions[user] = merge_intervals(intervals)

**⭐ 8. Time & Space Complexity**

Time Complexity: O(n log n) → sorting dominates

Space Complexity: O(n) → output list of merged intervals

**⭐ 9. Common Pitfalls & Mistakes**

❌ Modifying original intervals without sorting → can miss overlaps
❌ Comparing end with start using < instead of <= → misses touching intervals
✔ Always sort first
✔ Use max to extend current interval
✔ Return a new list, don’t mutate input unless necessary